In [1]:
import pandas as pd

In [2]:

df = pd.read_parquet('data.parquet')
cosine_sim = pd.read_parquet('cosine_sim.parquet').to_numpy()

In [3]:
df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,overview
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0,27.612,8.2,a mentally unstable vietnam war veteran works ...
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0,18.216,7.8,"king arthur, accompanied by his squire, recrui..."
3,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,392419.0,17.505,7.8,"brian cohen is an average young jewish man, bu..."
4,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,['horror'],['US'],NaN,tt0070047,8.1,391942.0,95.337,7.7,12-year-old regan macneil begins to adapt an e...
6,tm14873,Dirty Harry,MOVIE,When a madman dubbed 'Scorpio' terrorizes San ...,1971,R,102,"['thriller', 'crime', 'action']",['US'],NaN,tt0066999,7.7,153463.0,14.745,7.5,when a madman dubbed 'scorpio' terrorizes san ...


In [31]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from streamlit_lottie import st_lottie
import json
import numpy  as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests


def fetch_poster(movie_id):
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=020b311fe0559698373a16008dc6a672&language=en-US'.format(movie_id))
    data = response.json()
    print(data)
    return "https://image.tmdb.org/t/p/w500/" + data['poster_path']



def load_lottiefile(filepath: str):
    with open(filepath, "r") as f:
        return json.load(f)


df = pd.read_parquet('data.parquet')
cosine_sim = pd.read_parquet('cosine_sim.parquet').to_numpy()


indices = pd.Series(df.index, index=df["title"])
movie_list = df['title'].unique()

def get_recommendations(title, cosine_sim, top_k=5):
    
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:20]
    movie_indices = [i[0] for i in sim_scores if i[0] != idx]

    recommendations  = df.iloc[movie_indices] \
        .sort_values(["imdb_votes", "imdb_score"], ascending=False)[
            ["title", "description", "genres", "imdb_score", 'imdb_id']
        ] \
        .reset_index(drop=True)\
        .head(top_k)
  
    recommended_movies = []
    recommended_movies_posters = []
    for x in recommendations.title:
        try:
            movie_id = df[df['title']==x].imdb_id.unique()[0]
            recommended_movies.append(x)
            recommended_movies_posters.append(fetch_poster(movie_id))
        except:
            continue
    return recommended_movies, recommended_movies_posters

'A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action, attempting to save a preadolescent prostitute in the process.'

In [36]:
def fetch_poster(movie_id):
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=020b311fe0559698373a16008dc6a672&language=en-US'.format(movie_id))
    data = response.json()
    return "https://image.tmdb.org/t/p/w500/" + data['poster_path']

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,overview
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0,27.612,8.2,a mentally unstable vietnam war veteran works ...


In [113]:
df[['title']]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,overview
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0,27.612,8.2,a mentally unstable vietnam war veteran works ...
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0,18.216,7.8,"king arthur, accompanied by his squire, recrui..."
3,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,392419.0,17.505,7.8,"brian cohen is an average young jewish man, bu..."
4,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,['horror'],['US'],NaN,tt0070047,8.1,391942.0,95.337,7.7,12-year-old regan macneil begins to adapt an e...
6,tm14873,Dirty Harry,MOVIE,When a madman dubbed 'Scorpio' terrorizes San ...,1971,R,102,"['thriller', 'crime', 'action']",['US'],NaN,tt0066999,7.7,153463.0,14.745,7.5,when a madman dubbed 'scorpio' terrorizes san ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5800,tm1040816,Momshies! Your Soul is Mine,MOVIE,Three women with totally different lives accid...,2021,None,108,['comedy'],['PH'],NaN,tt14412240,5.8,26.0,4.112,NaN,three women with totally different lives accid...
5801,tm1014599,Fine Wine,MOVIE,A beautiful love story that can happen between...,2021,None,100,"['romance', 'drama']",['NG'],NaN,tt13857480,6.9,39.0,0.966,NaN,a beautiful love story that can happen between...
5802,tm1108171,Edis Starlight,MOVIE,Rising star Edis's career journey with ups and...,2021,None,74,"['music', 'documentation']",[],NaN,None,NaN,NaN,1.036,8.5,rising star edis's career journey with ups and...
5803,tm1045018,Clash,MOVIE,A man from Nigeria returns to his family in Ca...,2021,None,88,"['family', 'drama']","['NG', 'CA']",NaN,tt14620732,6.5,32.0,0.709,NaN,a man from nigeria returns to his family in ca...


In [118]:
genres_titles = df[['title', 'genres']]
genres_titles["genres"]=genres_titles["genres"].apply(utils.repair_array_bound_categories)
genres_titles = genres_titles.explode('genres').drop_duplicates()

<ipython-input-118-b182a4a18032>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [120]:
df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,overview
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0,27.612,8.2,a mentally unstable vietnam war veteran works ...
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0,18.216,7.8,"king arthur, accompanied by his squire, recrui..."
3,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,392419.0,17.505,7.8,"brian cohen is an average young jewish man, bu..."
4,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,['horror'],['US'],NaN,tt0070047,8.1,391942.0,95.337,7.7,12-year-old regan macneil begins to adapt an e...
6,tm14873,Dirty Harry,MOVIE,When a madman dubbed 'Scorpio' terrorizes San ...,1971,R,102,"['thriller', 'crime', 'action']",['US'],NaN,tt0066999,7.7,153463.0,14.745,7.5,when a madman dubbed 'scorpio' terrorizes san ...


In [87]:
lst = []

In [93]:
lsts =[]

In [94]:
for i in lst:
    lsts.extend(i)

TypeError: 'float' object is not iterable